In [ ]:
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt

In [ ]:
input_csv = "../results/openssl.csv"
input_sqlite_csv = "../results/sqlite.csv"
output_pdf = "../results/fig13.pdf"
baseline = "x86_64,qemu,qemu"
base_arch, base_runtime, base_tag = baseline.split(',')
df = pd.read_csv(input_csv, sep=';')
df

In [ ]:
# Select which ciphers to plot
ciphers = [ #'openssl.md5-64', 'openssl.md5-256',
            'openssl.md5-1024', 'openssl.md5-8192',
            #'openssl.rsa512-sign', 'openssl.rsa512-verify',
            'openssl.rsa1024-sign', 'openssl.rsa1024-verify',
            'openssl.rsa2048-sign', 'openssl.rsa2048-verify',
            #'openssl.sha1-64', 'openssl.sha1-256',
            'openssl.sha1-1024', 'openssl.sha1-8192',
            #'openssl.sha256-64', 'openssl.sha256-256',
            'openssl.sha256-1024', 'openssl.sha256-8192'
          ]

df = df.loc[df['bench'].isin(ciphers)]
df

In [ ]:
# Insert the sqlite results
df_sqlite = pd.read_csv(input_sqlite_csv, sep=';')
df_sqlite = df_sqlite.loc[df_sqlite['bench'] == "micro.sqlite-total-multi"]
df_sqlite['bench'] = "xxxxxxx.sqlite"
# convert to ops/s
df_sqlite['value'] = 269236 / (df_sqlite['value'] / 1000)
df_sqlite['unit'] = "ops/s"

df_sqlite

In [ ]:
df = pd.concat([ df, df_sqlite] , ignore_index=True)
df

In [ ]:
base_df = df.loc[df['tag'] == 'qemu']
base_df

In [ ]:
# Get the mean for each baseline benchmark
base_means = {}
for b in set(base_df['bench']):
    base_means[b] = np.array(base_df.loc[base_df['bench'] == b]['value'].values, dtype=np.float32).mean()

In [ ]:
# Print the mean of every benchmark for each runtime
mean_df = pd.DataFrame()
for b in sorted(set(df['bench'])):
    df_b = df.loc[df['bench'] == b]
    tmp_dict = { 'bench': b }
    for t in set(df_b['tag']):
        df_b_t = df_b.loc[df_b['tag'] == t]
        tmp_dict[t] = np.mean(df_b_t['value'])
    mean_df = mean_df.append(tmp_dict, ignore_index=True)
mean_df

In [ ]:
# Normalize all results from original df to these means
df_norm = pd.DataFrame(columns=['arch', 'bench', 'dataset', 'threads', 'unit', 'value', 'runtime',
                                'tag', 'norm', 'label'])
norm_vals = []
for row in df.itertuples():
    try:
        if row.arch == base_arch and row.runtime == base_runtime and row.tag == base_tag:
            continue
        #if row.bench == "sqlite.speedtest1":
         #   norm = base_means[row.bench] / float(row.value)      # speedup
        #else:
        norm = float(row.value) / base_means[row.bench]    # relative perf
        
        # norm = 100 * (base_means[row.bench] - float(row.value)) / base_means[row.bench]
        dct = row._asdict()
        dct['norm'] = norm
        dct['label'] = f"{dct['tag']}"
        # dct['label'] = f"{dct['runtime']}-{dct['tag']}"
        del dct['Index']
        del dct['cmdline']
        norm_vals.append(dct)
    except KeyError:
        pass
df_norm = df_norm.append(norm_vals, ignore_index=True)
df_norm

In [ ]:
import re 

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
fig = plt.figure(figsize=(5,2.5), dpi=500)
sbs.set(style="whitegrid")
palette = {
    'orange': '#faa200',
    'sky blue': '#00b7ec',
    'bluish green': '#00a077',
    'yellow': '#f5e636',
    'blue': '#0077b8',
    'vermillion': '#f3640d',
    'reddish purple': '#e47ead'
}
ax = sbs.barplot(data=df_norm, x='bench', y='norm', hue='label',
                 hue_order=['risotto', 'native'], order=sorted_nicely(base_means))
plt.grid(visible=True, axis='y')
plt.xticks(ticks=range(0, len(set(df_norm['bench']))),
           labels=[ l[8:] for l in sorted_nicely(base_means) ],  # remove openssl. from the x ticks
           rotation=45, ha="right", fontsize='xx-small')
plt.xlabel("")
plt.ylabel("Speedup w.r.t. QEMU")
plt.ylim((0, 27))
plt.axhline(y=1, xmin=0, xmax=1, color='tomato', linewidth=2.5)
# Annotate the raw value of the baseline
for idx, value in enumerate(sorted_nicely(set(base_means))):
    if base_means[value] > 1000000:
        v = f"{base_means[value] / 1000000:.0f}M"
    elif base_means[value] > 1000:
        v = f"{base_means[value] / 1000:.0f}k"
    else:
        v = f"{base_means[value]:.0f}"
    tmp = max(mean_df.loc[mean_df['bench'] == value].values[0][1:]) / base_means[value]
    plt.text(idx, tmp+1, f"{v}", fontsize='xx-small', color='tomato', rotation=90, ha='center')
    #plt.text(idx, max(tmp+.3, 21), f"{v}", fontsize='xx-small', color='tomato', rotation=45, ha='center')
#plt.vlines(range(0, len(set(base_means))), ymin=0, ymax=20.8, linestyle='dashed', colors='grey', linewidth=.5, zorder=0)

# Set color + hatch
style = {
    'fill': [ True, True ],
    'color': [ palette['bluish green'], palette['orange'] ],
    'hatch': [ '', '', ''],
    'edgecolor': [ 'black', 'black', 'black' ]
}
for idx, bar in enumerate(ax.patches):
    bar_nr = int(idx / int(len(base_means)))
    bar.set(color=style['color'][bar_nr], fill=style['fill'][bar_nr],
            hatch=style['hatch'][bar_nr], edgecolor=style['edgecolor'][bar_nr])

plt.legend(loc='upper center', bbox_to_anchor=(.5, 1.15), borderaxespad=0, ncol=2, fontsize='x-small')

In [ ]:
fig.savefig(fig13.pdf, dpi=500, bbox_inches='tight')

In [ ]:
for i in sorted_nicely(base_means):
    print(f"{i:25}: {base_means[i]:>20.1f}")

In [ ]:
print(value)
max(mean_df.loc[mean_df['bench'] == value].values[0][1:]) / base_means[value]

In [ ]:
mean_df['risotto-speedup'] = mean_df['risotto'] / mean_df['qemu']
mean_df